In [8]:
import numpy as np
import pandas as pd
import os

from pathlib import Path
import matplotlib.pyplot as plt
import cv2
from sklearn import *
from torchvision import transforms
import PIL.Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint

In [9]:
# for data frame columns name and values

Image_link=[]
Image_Name=[]
Image_label=[]

for root,ditrs,file in os.walk('/kaggle/input','weed-detection-in-soybean-crops'):
    for files in file:
        image_fl=os.path.join(root,files)
        #image_data=cv2.imread(image_fl)
        image_nm=image_fl.split('/')[5] 
         
        
        Image_link.append(image_fl)
        Image_Name.append(image_nm)

In [10]:
# Image labeling

Image_label=preprocessing.LabelEncoder().fit_transform(Image_Name)

In [11]:
# Create a data frame

df=pd.DataFrame(list(zip(Image_link,Image_Name,Image_label)),columns=['ImagePath','ImageName','ImageLabel'])

In [12]:
# separet weed and soybean data frame

df_weed=df[df.ImageName!='soybean']
df_soybean=df[df.ImageName=='soybean']
print('weed DataFrame shape :',df_weed.shape,'Soybean DataFrame shape :',df_soybean.shape)

In [13]:
# prepare to train only weed dataset

x=df_weed.ImagePath
y=df_weed.ImageLabel
xtrain,xtest,ytrain,ytest=model_selection.train_test_split(x,y,test_size=0.01,random_state=101)

In [14]:
# create image to array data

X_train=[]
for i in xtrain:
    img_pre=cv2.imread(str(i))
    img_pre=cv2.resize(img_pre, (150, 150))
    X_train.append(img_pre)
    
X_train=np.array(X_train)
ytrain=to_categorical(ytrain)

In [15]:
expecion0=Xception(weights='imagenet',include_top = False, input_shape=(150, 150, 3)) 


model = expecion0.output
model = tf.keras.layers.GlobalAveragePooling2D()(model)
model = tf.keras.layers.Dropout(0.5)(model)

model = tf.keras.layers.Dense(4, activation = "softmax")(model)

model = tf.keras.models.Model(inputs = expecion0.input , outputs = model)

In [16]:
model.compile(loss = 'categorical_crossentropy', 
             optimizer = "Adam", 
              metrics=["accuracy"]
             )

In [17]:
tensorboard = TensorBoard(log_dir="logs")
checkpoint = ModelCheckpoint("efficient.h5", 
                            monitor='val_accuracy', 
                            mode = "auto", 
                            verbose = 1,
                            save_best_only =True)
reduce_LR = ReduceLROnPlateau(monitor="val_accuracy", 
                             factor = 0.3, 
                             patience = 2, 
                             min_delta=0.001, 
                             mode = "auto", 
                            verbose = 1)

In [18]:
# fit only weed data and after fit, predict soybean data/main data for test if accuracy is above 95-99%

history = model.fit(X_train, ytrain,
                  validation_split = 0.1, 
                  epochs = 10, 
                  verbose =1, 
                  batch_size = 32, 
                  callbacks = [tensorboard, checkpoint, reduce_LR])